In [ ]:
import time
from typing import Dict, List
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import os


# ---------- CONFIG ----------
UA = {"User-Agent": "Giulia Petrilli giuliapetrilli2000@gmail.com"}
BASE = "https://data.sec.gov/api"

# --- TAG GROUPS ---
REVENUE_TAGS = [
    "RevenueFromContractWithCustomerExcludingAssessedTax",
    "Revenues",
    "SalesRevenueNet"
]
GROSS_PROFIT_TAGS = ["GrossProfit"]
COST_TAGS = ["CostOfRevenue", "CostOfGoodsAndServicesSold"]
NET_INCOME_TAGS = ["NetIncomeLoss"]

OPERATING_EXP_TAGS = ["OperatingExpenses", "SellingGeneralAndAdministrativeExpenses"]
R_AND_D_TAGS = ["ResearchAndDevelopmentExpense"]
DEPR_AMORT_TAGS = ["DepreciationAndAmortization", "AmortizationExpense"]
INTEREST_EXP_TAGS = ["InterestExpense"]
INTEREST_INC_TAGS = ["InterestIncome"]
INCOMEBEFORETAX_TAGS = ["IncomeBeforeTax"]
TAX_TAGS = ["IncomeTaxExpenseBenefit", "ProvisionForIncomeTaxes"]

EPS_BASIC_TAGS = ["EarningsPerShareBasic"]
EPS_DILUTED_TAGS = ["EarningsPerShareDiluted"]
SHARES_BASIC_TAGS = ["WeightedAverageNumberOfSharesOutstandingBasic"]
SHARES_DILUTED_TAGS = ["WeightedAverageNumberOfDilutedSharesOutstanding"]

ASSETS_TAGS = ["Assets"]
ASSETS_CURR_TAGS = ["AssetsCurrent"]
LIAB_TAGS = ["Liabilities"]
LIAB_CURR_TAGS = ["LiabilitiesCurrent"]
EQUITY_TAGS = ["StockholdersEquity", "Equity"]
CASH_TAGS = ["CashAndCashEquivalentsAtCarryingValue"]
RECEIVABLES_TAGS = ["AccountsReceivableNetCurrent"]
INVENTORY_TAGS = ["InventoriesNet"]
DEBT_TAGS = ["LongTermDebt"]

CAPEX_TAGS = ["CapitalExpenditures"]
OPER_CASHFLOW_TAGS = ["NetCashProvidedByOperatingActivities"]
INV_CASHFLOW_TAGS = ["NetCashUsedForInvestingActivities"]
FIN_CASHFLOW_TAGS = ["NetCashProvidedByFinancingActivities"]

EMPLOYEE_TAGS = ["NumberOfEmployees", "WeightedAverageNumberOfEmployees"]

# ---------- UTIL / FETCH ----------
def load_ticker_map() -> Dict[str, str]:
    url = "https://www.sec.gov/files/company_tickers.json"
    r = requests.get(url, headers=UA, timeout=30)
    r.raise_for_status()
    j = r.json()
    return {v["ticker"].upper(): f'{int(v["cik_str"]):010d}' for v in j.values()}

def _get_json(url: str):
    r = requests.get(url, headers=UA, timeout=30)
    if r.status_code == 404:
        return None
    r.raise_for_status()
    return r.json()

def company_concept(cik10: str, taxonomy: str, tag: str):
    url = f"{BASE}/xbrl/companyconcept/CIK{cik10}/{taxonomy}/{tag}.json"
    return _get_json(url)

def concept_to_df(j: dict, prefer_units=("USD", "USD$", "USD (in millions)")) -> pd.DataFrame:
    if j is None:
        return pd.DataFrame(columns=["fy","fp","end","val"])
    units = j.get("units", {})
    unit_key = None
    for u in prefer_units:
        if u in units:
            unit_key = u
            break
    if unit_key is None and units:
        unit_key = next(iter(units))
    rows = units.get(unit_key, [])
    if not rows:
        return pd.DataFrame(columns=["fy","fp","end","val"])
    df = pd.DataFrame(rows)
    for c in ("fy","fp","end","val"):
        if c not in df.columns: df[c] = pd.NA
    df["fy"] = pd.to_numeric(df["fy"], errors="coerce").astype("Int64")
    df["end"] = pd.to_datetime(df["end"], errors="coerce")
    return df.sort_values(["end","fy"], na_position="last").reset_index(drop=True)[["fy","fp","end","val"]]

def quarterly_series(df: pd.DataFrame) -> pd.DataFrame:
    q = df[df["fp"].str.upper().str.startswith("Q")].copy()
    if q.empty:
        return pd.DataFrame(columns=["fy","fp","end","value"])
    q["qnum"] = q["fp"].str.upper().str.extract(r"Q(\d)").astype("Int64")
    q = (q.sort_values("end")
           .groupby(["fy","qnum"], dropna=True)
           .agg(value=("val","last"), end=("end","last"))
           .reset_index())
    q["fp"] = "Q" + q["qnum"].astype("Int64").astype(str)
    return q[["fy","fp","end","value"]].sort_values(["fy","fp"])

def _fetch_first_available(cik10: str, tags: List[str], label: str = "") -> pd.DataFrame:
    for tag in tags:
        j = company_concept(cik10, "us-gaap", tag)
        if j is None:
            continue
        df = concept_to_df(j)
        if not df.empty:
            print(f"[{label}] Using tag: {tag}")
            return df
        time.sleep(0.2)
    print(f"[{label}] No data for tags: {tags}")
    return pd.DataFrame(columns=["fy","fp","end","val"])

def annual_series(df: pd.DataFrame) -> pd.DataFrame:
    """Extract annual datapoints from SEC fact dataframe."""
    if df.empty:
        return pd.DataFrame(columns=["fy", "fp", "end", "value"])

    annual_flags = ["FY", "Y", "CY", "ANN", "ANNUAL"]

    a = df[df["fp"].str.upper().isin(annual_flags)].copy()
    if a.empty:
        return pd.DataFrame(columns=["fy", "fp", "end", "value"])

    a = (
        a.sort_values("end")
         .groupby("fy", dropna=True)
         .agg(value=("val", "last"), end=("end", "last"))
         .reset_index()
    )
    a["fp"] = "FY"

    return a[["fy", "fp", "end", "value"]].sort_values("fy")


# ---------- MAIN FUNCTION ----------
def get_quarterly_financials(ticker: str, last_n_quarters: int = 16) -> pd.DataFrame:
    tmap = load_ticker_map()
    tk = ticker.upper()
    if tk not in tmap:
        raise ValueError(f"Ticker {ticker} not in SEC mapping.")
    cik10 = tmap[tk]

def _add_column_both(cik10, tags, colname, label=""):
    raw = _fetch_first_available(cik10, tags, label=label)
    q = quarterly_series(raw).rename(columns={"value": colname})
    a = annual_series(raw).rename(columns={"value": colname})
    q["frequency"] = "Q"
    a["frequency"] = "A"
    return q, a


def get_financials(ticker: str, last_n_quarters: int = 16) -> pd.DataFrame:
    tmap = load_ticker_map()
    cik10 = tmap[ticker.upper()]

    quarts = []
    annuals = []

    def add(tags, col, label):
        q, a = _add_column_both(cik10, tags, col, label)
        quarts.append(q)
        annuals.append(a)

    # ---- Core metrics ----
    add(REVENUE_TAGS, "revenue", "Revenue")
    add(GROSS_PROFIT_TAGS, "gross_profit", "Gross Profit")
    add(NET_INCOME_TAGS, "net_income", "Net Income")
    add(OPERATING_EXP_TAGS, "operating_expenses", "Operating Exp")
    add(R_AND_D_TAGS, "r_and_d", "R&D")
    add(DEPR_AMORT_TAGS, "depr_amort", "D&A")
    add(INTEREST_EXP_TAGS, "interest_expense", "Interest Exp")
    add(INTEREST_INC_TAGS, "interest_income", "Interest Inc")
    add(INCOMEBEFORETAX_TAGS, "income_before_tax", "Income Before Tax")
    add(TAX_TAGS, "tax_expense", "Tax Expense")
    add(EPS_BASIC_TAGS, "eps_basic", "EPS Basic")
    add(EPS_DILUTED_TAGS, "eps_diluted", "EPS Diluted")
    add(SHARES_BASIC_TAGS, "shares_basic", "Shares Basic")
    add(SHARES_DILUTED_TAGS, "shares_diluted", "Shares Diluted")
    add(ASSETS_TAGS, "assets", "Assets")
    add(ASSETS_CURR_TAGS, "assets_current", "Assets Current")
    add(LIAB_TAGS, "liabilities", "Liabilities")
    add(LIAB_CURR_TAGS, "liabilities_current", "Liabilities Current")
    add(EQUITY_TAGS, "equity", "Equity")
    add(CASH_TAGS, "cash", "Cash")
    add(RECEIVABLES_TAGS, "accounts_receivable", "AR")
    add(INVENTORY_TAGS, "inventory", "Inventory")
    add(DEBT_TAGS, "long_term_debt", "Debt")
    add(CAPEX_TAGS, "capex", "CapEx")
    add(OPER_CASHFLOW_TAGS, "operating_cf", "Op Cash Flow")
    add(INV_CASHFLOW_TAGS, "investing_cf", "Inv Cash Flow")
    add(FIN_CASHFLOW_TAGS, "financing_cf", "Fin Cash Flow")
    add(EMPLOYEE_TAGS, "employees", "Employees")
    # ------ Merge Quarterly ------
    df_q = quarts[0]
    for d in quarts[1:]:
        df_q = df_q.merge(d, on=["fy", "fp", "end", "frequency"], how="outer")
    # ------ Merge Annual ------
    df_a = annuals[0]
    for d in annuals[1:]:
        df_a = df_a.merge(d, on=["fy", "fp", "end", "frequency"], how="outer")
    # ------ Compute margins ------
    for df in (df_q, df_a):
        df["gross_margin_pct"] = df["gross_profit"] / df["revenue"]
        df["net_margin_pct"]   = df["net_income"]  / df["revenue"]
    # Keep only last N quarters for quarterly
    df_q = df_q.sort_values("end")
    if last_n_quarters:
        df_q = df_q.tail(last_n_quarters)
    # add 'start' column with YYYY-01-01 for annual (FY) rows, NaT for others
    df['start'] = pd.NaT 
    fy_mask = df['fp'].astype(str).str.upper() == 'FY'
    df.loc[fy_mask, 'start'] = pd.to_datetime(df.loc[fy_mask, 'fy'].astype(str) + '-01-01', errors='coerce')
    # ------ Combine into one long-format DF ------
    df = pd.concat([df_q, df_a], ignore_index=True).sort_values(["fy", "fp", "end", "start", "frequency"])
    return df.reset_index(drop=True)

In [ ]:

if __name__ == "__main__":
    # --- USER INPUTS ---
    boycotted = True  # set to True for boycott targets, False for control group
    tickers = [
    "MCD",     # McDonald's (NYSE)
]

 # <--- add any tickers you want here
    last_n_quarters = 200

    base_path = "/Users/giuliamariapetrilli/Documents/GitHub/masters_thesis/data"

    # --- LOOP THROUGH TICKERS ---
    for TICKER in tickers:
        print(f"\n===== Processing {TICKER.upper()} =====")

        try:
            # Fetch data
            df = get_financials(TICKER, last_n_quarters=last_n_quarters)
            if df.empty:
                print(f"⚠️ No data returned for {TICKER}. Skipping.")
                continue

            # Add metadata columns
            df.insert(0, 'ticker', TICKER.upper())
            df.insert(0, 'boycotted', int(boycotted))

            # Choose folder based on flag
            if boycotted:
                folder = f"{base_path}/boycott_target/{TICKER.lower()}"
            else:
                folder = f"{base_path}/control_group/{TICKER.lower()}"

            # Create folder if needed
            os.makedirs(folder, exist_ok=True)

            # Save file
            output_path = f"{folder}/{TICKER.lower()}_quarterly.csv"
            df.to_csv(output_path, index=False)

            # Show confirmation
            print(f"✅ Saved {TICKER} data to: {output_path}")
            print(f"   → Shape: {df.shape}")
            print(f"   → Columns: {list(df.columns)}")

        except Exception as e:
            print(f"❌ Error processing {TICKER}: {e}")


In [ ]:
# check if the df gets generated
if __name__ == "__main__":
    TICKER = "MCD"  # replace with any ticker
    df = get_financials(TICKER, last_n_quarters=200)

    print(df)

    # Show all columns and shape
    print(df.columns)
    print(df.shape)

In [ ]:
import matplotlib.pyplot as plt

# Sort by date and split the dataframe
df_sorted = df.sort_values("end")
df_q = df_sorted[df_sorted["frequency"] == "Q"]
df_a = df_sorted[df_sorted["frequency"] == "A"]

plt.figure(figsize=(10,5))

# Quarterly line
plt.plot(df_q["end"], df_q["revenue"], marker="o", label="Quarterly (Q)")

# Annual line
plt.plot(df_a["start"], df_a["revenue"], marker="s", label="Annual (A)")

plt.title("Revenue Over Time (Quarterly vs Annual)")
plt.xlabel("Date")
plt.ylabel("Revenue (USD)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
 # Show all columns and shape
print(df_a.columns)
print(df_a.shape)

output_path = f"{'/Users/giuliamariapetrilli/Documents/GitHub/masters_thesis/data'}/{TICKER.lower()}_annual.csv"
output_path_compare = f"{'/Users/giuliamariapetrilli/Documents/GitHub/masters_thesis/data'}/{TICKER.lower()}_quarter_compare.csv"
df_a.to_csv(output_path, index=False)
df_q.to_csv(output_path_compare, index=False)

In [ ]:
# single company

if __name__ == "__main__":
    boycotted = False # set to False for control group
    TICKER = "BROS"  # replace with any ticker
    df = get_quarterly_financials(TICKER, last_n_quarters= 200)

    print(df)

    df.insert(0, 'ticker', TICKER.upper())

    if boycotted:
        df.insert(0, 'boycotted', 1)
    else:
        df.insert(0, 'boycotted', 0)

    # Choose folder based on boycotted flag
    base_path = "/Users/giuliamariapetrilli/Documents/GitHub/masters_thesis/data"
    if boycotted:
        folder = f"{base_path}/boycott_target/{TICKER.lower()}"
    else:
        folder = f"{base_path}/control_group/{TICKER.lower()}"

    # Create folder if it doesn’t exist
    os.makedirs(folder, exist_ok=True)

    # Save the file
    output_path = f"{folder}/{TICKER.lower()}_quarterly.csv"
    df.to_csv(output_path, index=False)

    # Show all columns and shape
    print(df.columns)
    print(df.shape)

    print(f"✅ File saved to: {output_path}")
